# Fine-tuning BERT model for Binary Text Classification

## Preparation for Google Collab

In [ ]:
import torch, os
from google.colab import drive

drive.mount("/content/drive")
os.listdir()
os.chdir("drive/MyDrive/akademi/Packt NLP with Transformers/CH05")

In [ ]:
!pip install transformers datasets

In [ ]:
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"

### Loading pre-trained model and tokenizer

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(
    model_path, id2label={0: "NEG", 1: "POS"}, label2id={"NEG": 0, "POS": 1}
)

## Loading popular IMDB dataset

In [ ]:
from datasets import load_dataset

# to take entire dataset from original train 25 K AND TEST 25K
"""
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

"""
# to take smaller portion 4K for train, 1K for test and 1K for validation
imdb_train = load_dataset("imdb", split="train[:2000]+train[-2000:]")
imdb_test = load_dataset("imdb", split="test[:500]+test[-500:]")
imdb_val = load_dataset("imdb", split="test[500:1000]+test[-1000:-500]")

In [ ]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

In [ ]:
enc_train = imdb_train.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_test = imdb_test.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_val = imdb_val.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)

In [ ]:
import pandas as pd

pd.DataFrame(enc_train)

## Preparing training settings with TrainingArguments and Trainer class

TrainingArguments is the subset of the arguments we use in our example scripts **which relate to the training loop
itself**.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./MyIMDBModel",
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    # TensorBoard log directory
    logging_dir="./logs",
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=cuda.is_available(),
    load_best_model_at_end=True,
)

Let's design our evaluation metrics as follows:

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1, "Precision": precision, "Recall": recall}

In [ ]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
    # training arguments that we defined above
    args=training_args,
    # training and validation dataset
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics=compute_metrics,
)

Trainer class is an optimized tool for Transformer to organize training and evalulation process for PyTorch and Tensorflow.

In [ ]:
results = trainer.train()

* For small portion

We have 4K training examples and the batch size is 32, which means each epoch takes 4k/32 = 125 steps. For 3 epoc, this will have 375 steps as wee see in the output.

* For full dataset

We have 25K training  examples with the batch size pf 32, so we will have 25K/32 ~=782 (roughly) steps. for 3 epoch we will have 2346 steps to run as seen in the progress bar 

Trainer model keeps the best model at the end. Lets evaluate the model on the train/test/validation

In [ ]:
q = [trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train", "val", "test"]).iloc[:, :5]

In [ ]:
# saving the best fine-tuned model & tokenizer
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
def get_prediction(text):
    inputs = tokenizer(
        text, padding=True, truncation=True, max_length=250, return_tensors="pt"
    ).to(device)
    outputs = model(inputs["input_ids"].to(device), inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [ ]:
model.to(device)
text = "I didn't like the movie since it bored me "
get_prediction(text)[1].item()

Use the model with pipeline 

In [ ]:
from transformers import (
    pipeline,
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
)

model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")
tokenizer = DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
nlp("the movie was very impressive")

In [ ]:
nlp("the script of the picture was very poor")